In [1]:
%pip install scipy
%pip install matplotlib
%pip install ipywidgets
%pip install numpy

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [44]:
import matplotlib.pyplot as plt
from ipywidgets import Layout, interact, FloatSlider, FloatLogSlider, IntSlider

def plot_circuit(solution):
    fig, ax1 = plt.subplots(figsize=(8, 4))
    # Plot current
    ax1.plot(solution['time'], solution['current'], 'r', label='Current (A)')
    ax1.set_xlabel('Time (s)')
    ax1.set_ylabel('Current (A)', color='r')
    ax1.tick_params(axis='y', labelcolor='r')
    ax1.grid(True)
    # ax2 = ax1.twinx()
    # ax2.plot(solution.time, solution.acceleration, 'b', label='B')
    # ax2.set_ylabel('Acceleration', color='b')
    # ax2.tick_params(axis='y', labelcolor='b')
    plt.show()

def plot_kinematics(output):
    fig, ax1 = plt.subplots(figsize=(8, 3))
    ax1.plot(output['time'], output['acceleration'], 'm', label='Acc (m/s^2)')
    ax1.set_xlabel('Time (s)')
    ax1.set_ylabel('Acc (m/s^2)', color='m')
    ax1.tick_params(axis='y', labelcolor='m')
    ax2 = ax1.twinx()
    ax2.plot(output['time'], output['velocity'], 'g', label='Velocity (m/s)')
    ax2.set_xlabel('Time (s)')  # Set x-axis label to seconds
    ax2.set_ylabel('Velocity (m/s)', color='g')
    ax2.tick_params(axis='y', labelcolor='g')
    plt.show()

def plot_position(output):
    fig, ax1 = plt.subplots(figsize=(8, 3))
    ax1.plot(output['time'], output['position'], 'b', label='Pos (m)')
    ax1.set_xlabel('Time (s)')
    ax1.set_ylabel('Pos (m)', color='m')
    ax1.tick_params(axis='y', labelcolor='m') 
    plt.show


def start(f):
    interact(
        f,
        V0=IntSlider(value=100, min=0, max=100000, step=5, description='V0 (Volts)'),
        C=FloatLogSlider(value=0.1, base=10, min=-6, max=2, step=0.1, description='C (Farads)'),
        R=FloatSlider(value=3, min=0.1, max=10, step=0.1, description='R'),
        N=IntSlider(value=100, min=1, max=1000, description='Turns'),
        D=FloatSlider(value=.01, min=.001, max=.1, step=0.001, description='Diameter of coil (m)', layout=Layout(width='400px'), style={'description_width': '250px'}),
        l=FloatSlider(value=.06, min=.01, max=.3, step=0.01, description='Length of coil (m)'),
        sd=FloatSlider(value=0.005, min=0.001, max=.1, step=0.001, description='Diameter of slug (m)'),
        sl=FloatLogSlider(value=0.010, base=10, min=-3, max=1, step=0.01, description='Length of slug (m)'),
        duration_s=FloatLogSlider(value=.2, base=10, min=-3, max=2, step=0.1, description='Duration')
    )


In [45]:
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, FloatLogSlider, IntSlider
from scipy.integrate import solve_ivp, cumulative_trapezoid
import numpy as np
from collections import namedtuple

mu_0 = 4 * np.pi * 1e-7  # Permeability of free space in T·m/A

# Projectile properties
proj_diameter = 0.005  # Diameter in meters
proj_length = 0.010  # Length in meters
iron_density = 7870  # Density of iron in kg/m^3

SimulationParameters = namedtuple('SimulationParameters',
                                  ['init_volts',
                                   'capacitance',
                                   'resistance',
                                   'num_turns',
                                   'coil_diameter',
                                   'coil_length',
                                   'slug_diameter',
                                   'slug_length',
                                   'duration_s',
                                  ])

# Define the differential equation for the RLC circuit
def circuit_model(t, y, params):
    # Destructure the state vector
    [voltage, current] = y

    # Rate of change in voltage
    dVdt = current / params.capacitance

    # Rate of change in current   # Compute inductance, L
    coil_area = np.pi * (params.coil_diameter / 2) ** 2  # Coil cross-section, in m^2
    L = (params.num_turns ** 2 * mu_0 * coil_area) / params.coil_length  # Inductance in Henrys  
    dIdt = -(params.resistance * current + voltage) / L

    # if count % 100 == 0:
    #     print(f't={t:.1e} I={current:.1e} B={B:.1e} F={F:.1e} a={a:.1e} ')
    return [dVdt, dIdt]

def force_model(current, params):
    B_vals = [];
    F_vals = [];
    a_vals = [];
    turns_density=params.num_turns/params.coil_length
    chi_iron = 1000
    slug_volume = np.pi * (params.slug_diameter/2) ** 2 * params.slug_length  # Volume in cubic meters
    slug_mass = iron_density * slug_volume  # Mass in kg
    for inst_current in current:
        B = mu_0 * turns_density * inst_current
        F = (chi_iron * B**2 * slug_volume)/(2*mu_0*(1+chi_iron)**2)
        a=F/slug_mass
        B_vals.append(B)
        F_vals.append(F)
        a_vals.append(a)
        # print(f'a={a:.6f}')
    return { 'mag_field':  np.array(B_vals), 
            'force':  np.array(F_vals), 
            'acceleration':  np.array(a_vals) }

time_step = 1e-6  # seconds
def run_sim(params):
    output={}
    t_span = (0, params.duration_s)  # Simulation time span
    t_eval = np.arange(0,params.duration_s, time_step)  # Set simulation time step
    # Solve the differential equation
    circuit_soln = solve_ivp(
        circuit_model,
        t_span,
        [params.init_volts, 0],
        t_eval=t_eval,
        args=(params,))
    output['time']=circuit_soln.t
    output['voltage']=circuit_soln.y[0]
    output['current']=circuit_soln.y[1]
    output = output | force_model(output['current'], simulationParameters)
    output['velocity'] = cumulative_trapezoid(output['acceleration'], output['time'], initial=0)
    output['position'] = cumulative_trapezoid(output['velocity'], output['time'], initial=0)   
    return output


def solve_and_plot(V0, C, R, N, D, l, sd, sl, duration_s):
    simulationParameters = SimulationParameters(init_volts=V0,
                                     capacitance=C,
                                     resistance=R,
                                     num_turns=N,
                                     coil_diameter=D,
                                     coil_length=l,
                                     slug_diameter=sd,
                                     slug_length=sl,
                                     duration_s=duration_s)
    output = run_sim(simulationParameters)
    plot_circuit(output)
    plot_kinematics(output)
    plot_position(output)

start(solve_and_plot)

#Test integration
# vel2 = cumulative_trapezoid(solution.acceleration, solution.time)
# Adjust the time array to match the length of the integrated array
# time_adjusted = solution.time[:-1]
# position = cumulative_trapezoid(solution.velocity, solution.time)
# plt.plot(time_adjusted, vel2, label='v(t)')
# plt.xlabel('Time (s)')
# plt.ylabel('V (m)/s')
# plt.legend()
# plt.show

interactive(children=(IntSlider(value=100, description='V0 (Volts)', max=100000, step=5), FloatLogSlider(value…